# Cloud-Net analysis
This notebook provides analysis of the Cloud-Net model results

In [1]:
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm

In [2]:
# put results in more readable format
df = pd.read_csv('metrics.csv')

df = df[['Scene ID','Threshold','Precision','Recall','F-1','Accuracy','Jaccard Index']]
df['Precision'] = (df['Precision'] * 100).round(4)
df['Recall'] = (df['Recall'] * 100).round(4)
df['F-1'] = (df['F-1'] * 100).round(4)
df['Jaccard Index'] = (df['Jaccard Index'] * 100).round(4)
df['Accuracy'] = (df['Accuracy'] * 100).round(4)
df['Threshold'] = df['Threshold'].round(6)

df.to_csv('metrics.tsv', index=False, sep='\t')

Average results:

In [3]:
avg_metrics = df.mean(numeric_only=True)
print("Average metrics:")
print(avg_metrics)

Average metrics:
Threshold         0.047059
Precision        75.503761
Recall           80.843288
F-1              74.632960
Accuracy         93.003004
Jaccard Index    67.553805
dtype: float64


Get the worst results

In [4]:
df = pd.read_csv('metrics.tsv', sep='\t')
min_f1 = df['F-1'].min()
min_accuracy = df['Accuracy'].min()

min_f1_results = df[df['F-1'] == min_f1]
min_accuracy_results = df[df['Accuracy'] == min_accuracy]

print("Results with minimal F1:")
print(min_f1_results)

print("Results with minimal Accuracy:")
print(min_accuracy_results)

Results with minimal F1:
                  Scene ID  Threshold  Precision  Recall  F-1  Accuracy  \
24   LC80980762014216LGN00   0.047059        0.0     0.0  0.0   99.9989   
27   LC81660432014020LGN00   0.047059        0.0     0.0  0.0   99.6741   
36   LC81460162014168LGN00   0.047059        0.0     0.0  0.0   97.3611   
50   LC80180082014215LGN00   0.047059        0.0     0.0  0.0   99.4191   
58   LC80420082013220LGN00   0.047059        0.0     0.0  0.0   99.6403   
60   LC80530022014156LGN00   0.047059        0.0     0.0  0.0   97.3932   
97   LC81750432013144LGN00   0.047059        0.0     0.0  0.0   77.7231   
101  LC81030162014107LGN00   0.047059        0.0     0.0  0.0   99.9096   

     Jaccard Index  
24             0.0  
27             0.0  
36             0.0  
50             0.0  
58             0.0  
60             0.0  
97             0.0  
101            0.0  
Results with minimal Accuracy:
                 Scene ID  Threshold  Precision   Recall      F-1  Accuracy  \


Get the best results

In [5]:
max_f1 = df['F-1'].max()
max_accuracy = df['Accuracy'].max()

max_f1_results = df[df['F-1'] == max_f1]
max_accuracy_results = df[df['Accuracy'] == max_accuracy]

print("Results with maximal F1:")
print(max_f1_results)

print("Results with maximal Accuracy:")
print(max_accuracy_results)

Results with maximal F1:
                                    Scene ID  Threshold  Precision   Recall  \
78  LC08_L1TP_066014_20160520_20170223_01_T1   0.047059     99.956  99.9995   

        F-1  Accuracy  Jaccard Index  
78  99.9777   99.9754        99.9555  
Results with maximal Accuracy:
                 Scene ID  Threshold  Precision  Recall  F-1  Accuracy  \
24  LC80980762014216LGN00   0.047059        0.0     0.0  0.0   99.9989   

    Jaccard Index  
24            0.0  


Sort scenes by f1

In [6]:
sorted_df = df.sort_values(by='F-1', ascending=False)
scene_ids = sorted_df[['Scene ID', 'F-1', 'Accuracy']]
scene_ids.reset_index(drop=True, inplace=True)
scene_ids

,Scene ID,F-1,Accuracy
0,LC08_L1TP_066014_20160520_20170223_01_T1,99.9777,99.9754
1,LC81510262014139LGN00,99.9735,99.9641
2,LC81220422014096LGN00,99.9621,99.9485
3,LC80290292014132LGN00,99.9578,99.9411
4,LC80170312013157LGN00,99.9574,99.9405
...,...,...,...
111,LC81460162014168LGN00,0.0000,97.3611
112,LC80180082014215LGN00,0.0000,99.4191
113,LC80530022014156LGN00,0.0000,97.3932
114,LC81030162014107LGN00,0.0000,99.9096


In [10]:
from evaluation import get_patches_for_sceneid
from tiff_utils import reassemble_band_patches

root = '/home/jakub.suran/netstore1/Cloud-Net_evaluation/'
gt_dir = os.path.join(root, 'Entire_scene_gts')
colors = ['blue', 'green', 'red']

for color in colors:
    print("Processing band:", color)
    target_dir = os.path.join(root, f'full_scene_{color}')
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    patch_dir = os.path.join(root, f'test_{color}')
    total_pathes = 0
    # Loop through each scene ID to stitch patches
    for _, (scene_id, f1, accuracy) in tqdm(scene_ids.iterrows(), "Combining pathes..."):
        gt_path = os.path.join(gt_dir, f'edited_corrected_gts_{scene_id}.TIF')
        gt = Image.open(gt_path)
        
        patches = get_patches_for_sceneid(patch_dir, scene_id)
        total_pathes += len(patches)
        
        # Reassemble patches to a single image
        reassemble_band_patches(patches, 384, 384, gt.size, f'{target_dir}/full_{color}_{scene_id}.TIF')

    print(f"Processed {total_pathes} patches.")

Processing band: blue


Combining pathes...: 9it [02:02, 13.58s/it]


KeyboardInterrupt: 